# Procesador de Sanciones IMSS Bienestar

In [1]:
# Importa librerías, chromedriver y carpeta de trabajo
import sys
import os
import glob
import yaml
import pandas as pd
from datetime import datetime
import shutil
# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (whee Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
Libreria_sancionesIMSSB = os.path.join(script_folder, "Libreria_SancionesIMSSB")
if Libreria_sancionesIMSSB not in sys.path:
    sys.path.append(Libreria_sancionesIMSSB)
Libreria_comunes = os.path.join(script_folder, "Libreria_comunes")
if Libreria_comunes not in sys.path:
    sys.path.append(Libreria_comunes)

sanciones_folder = os.path.join(folder_root, "Implementación", "Sanciones IMSSB")
from chrome_driver_load import load_chrome


In [2]:
# Obtén folders sin acuse y define cuál vas a procesar
from sanciones_IMSSB_processing import get_folders_pending, print_columns

pending = get_folders_pending(sanciones_folder)
print("📂 Folders sin _acuse.pdf_:\n")
print_columns(pending, n_cols=2)
# Solicita al usuario una carpeta válida de la lista `pending`
while True:
    processing_folder = input("Please input the folder to process: ")
    if processing_folder in pending:
        processing_folder = os.path.join(folder_root, "Implementación", "Sanciones IMSSB",f"{processing_folder}")
        print(f"\n📂 Folder a procesar: {os.path.basename(processing_folder)}")
        
        break
    print(f"'{processing_folder}' no está en la lista de pendientes, intenta de nuevo.")


📂 Folders sin _acuse.pdf_:

CCPA-4994                          INSABI-UCNAF-CPP-DF-AS-5709-2023   
DPS-AS-02563-2025                  INSABI-UCNAF-CPP-DF-AS-6036-2023   
DPS-AS-02564-2025                  INSABI-UCNAF-CPP-DF-AS-7323-2023   
INSABI-UCNAF-CPP-DF-AS-10190-2023  INSABI-UCNAF-CPP-DF-AS-7362-2023   
INSABI-UCNAF-CPP-DF-AS-10192-2023  Oficio 00132                       
INSABI-UCNAF-CPP-DF-AS-5704-2023   


Please input the folder to process:  DPS-AS-02563-2025



📂 Folder a procesar: DPS-AS-02563-2025


## Generación de relación, limpieza de nombres.

### Genera relación

In [4]:
# Genera resumen, confirma que _relacion tiene columnas necesarias, extrae fechas
from sanciones_IMSSB_folders_completos import folders_completos
folders_completos(sanciones_folder)


*******************************
Oficios con excel: ['CCPA-00695', 'CCPA-01726', 'CCPA-02199', 'CCPA-03424', 'CCPA-3647', 'CCPA-4994', 'CDS-1205', 'COA-00149', 'COA-00906', 'COA-01190', 'COA-01219', 'COA-02804', 'COA-03069', 'COA-03071', 'COA-03225', 'COA-03359-1', 'COA-03598', 'COA-03833', 'COA-04724', 'COA-05493', 'COA-05534', 'COA-06065', 'COA-08146', 'COA-08698', 'COA-08699', 'COA-09042', 'COA-09541', 'COA-10090', 'COA-10093', 'COA-10816', 'COA-10955', 'COA-11001', 'COA-11003', 'COA-11090', 'COA-11726', 'COA-2290', 'COA-7751', 'DPS-AS-02563-2025', 'DPS-AS-02564-2025', 'INSABI-UCNAF-CPP-DF-AS-10190-2023', 'INSABI-UCNAF-CPP-DF-AS-10192-2023', 'INSABI-UCNAF-CPP-DF-AS-5704-2023', 'INSABI-UCNAF-CPP-DF-AS-5709-2023', 'INSABI-UCNAF-CPP-DF-AS-6036-2023', 'INSABI-UCNAF-CPP-DF-AS-7323-2023', 'INSABI-UCNAF-CPP-DF-AS-7362-2023', 'Oficio 00132', 'Oficio 00476', 'Oficio 00797', 'Oficio 00798', 'Oficio 01265']
Exceles válidos: ['CCPA-00695_relacion.xlsx', 'CCPA-01726_relacion.xlsx', 'CCPA-02199_r

### Limpieza de nombres

In [5]:
# Limpia los nombres de las NC
from Sanciones_IMSSB_clean import clean_names
clean_names(processing_folder)

## Procesamiento de XML's y generación de wordpads

In [ ]:
# Aplica estructura a los XML y cambia a PUE
from sanciones_IMSSB_XMLS import pretty_xml_files, XML_PUE
pretty_xml_files(processing_folder)
XML_PUE(processing_folder)

In [ ]:
# Transforma los XML's a wordpads
from Sanciones_IMSSB_XMLwordpad import xml_to_wordpad
xml_to_wordpad(processing_folder)

## Procesamiento y generación de PDF's

In [ ]:
# Cambia manualmente los PDF's a PUE
from sanciones_IMSSB_PDFsaPUESelectivo import PDFsaPUE
string_to_change= "PUE - Pago en una sola exhibición"
PDFsaPUE(processing_folder, string_to_change)

In [ ]:
# Descarga los _SAT de los QRS
from sanciones_IMSSB_processing_QRS_Total import descargaQRSs
chrome_driver = load_chrome(processing_folder)
descargaQRSs(processing_folder, chrome_driver)

## Auditoría previa a la generación de entregables

In [ ]:
# Generación de auditoría
from Sanciones_IMSSB_extraeauditoriaUUIDNC import auditarentregables
auditarentregables(processing_folder)

## Generación de entregables

In [ ]:
# Generar excel INSABI
from sanciones_IMSSB_Generar_FormatoINSABI import GenerarFormatoINSABI
json_path = os.path.join(folder_root, "Implementación", "key.json")
template_file = os.path.join(folder_root, "Implementación", "Templates", "TemplateIMSSB.xlsx")
GenerarFormatoINSABI(processing_folder, json_path, template_file)

In [ ]:
# Generar ZIP
from sanciones_IMSSB_genera_zip import genera_zip
genera_zip(processing_folder)

In [ ]:
# Generar archivo para imprimir
from sanciones_IMSSB_PDFprint import PDF_para_imprimir
PDF_para_imprimir(processing_folder)